In [18]:
import sys
sys.path.append("modules/")

import numpy as np
import pandas
import mnist_loader
import random
import network2 as network_basic
import network2_L1 as network_L1
import time
import matplotlib.pyplot as plt

In [2]:
# Load data
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [3]:
# Let's do unregularized run
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = network_basic.Network([784, 30, 10], cost=network_basic.CrossEntropyCost)
unreg_run = net.SGD(
    training_data, 30, 10, 0.5,
    lmbda = 0.,
    evaluation_data=validation_data,
    monitor_evaluation_accuracy=True,
    monitor_evaluation_cost=True,
    monitor_training_accuracy=True,
    monitor_training_cost=True
       )

Epoch 0 training complete
Cost on training data: 0.398932790182
Accuracy on training data: 47061 / 50000
Cost on evaluation data: 0.394579805499
Accuracy on evaluation data: 9418 / 10000

Epoch 1 training complete
Cost on training data: 0.321863001622
Accuracy on training data: 47716 / 50000
Cost on evaluation data: 0.330295003356
Accuracy on evaluation data: 9519 / 10000

Epoch 2 training complete
Cost on training data: 0.307523345699
Accuracy on training data: 47875 / 50000
Cost on evaluation data: 0.339940302832
Accuracy on evaluation data: 9540 / 10000

Epoch 3 training complete
Cost on training data: 0.277162687446
Accuracy on training data: 48045 / 50000
Cost on evaluation data: 0.317909887073
Accuracy on evaluation data: 9571 / 10000

Epoch 4 training complete
Cost on training data: 0.273384680907
Accuracy on training data: 48137 / 50000
Cost on evaluation data: 0.330007670737
Accuracy on evaluation data: 9548 / 10000

Epoch 5 training complete
Cost on training data: 0.248875509

In [34]:
# Let's see if we can outperform that with L1 (note that I'm uing network_L1 module)
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = network_L1.Network([784, 30, 10], cost=network_L1.CrossEntropyCost)

# lmbda = 1
L1_run_lmbda_1 = net.SGD(
    training_data, 30, 10, 0.5,
    lmbda = 1.,
    evaluation_data=validation_data,
    monitor_evaluation_accuracy=True,
    monitor_evaluation_cost=True,
    monitor_training_accuracy=True,
    monitor_training_cost=True
       )

#lmbda = 2
L1_run_lmbda_2 = net.SGD(
    training_data, 30, 10, 0.5,
    lmbda = 2.,
    evaluation_data=validation_data,
    monitor_evaluation_accuracy=True,
    monitor_evaluation_cost=True,
    monitor_training_accuracy=True,
    monitor_training_cost=True
       )

#lmbda = 3
L1_run_lmbda_3 = net.SGD(
    training_data, 30, 10, 0.5,
    lmbda = 3.,
    evaluation_data=validation_data,
    monitor_evaluation_accuracy=True,
    monitor_evaluation_cost=True,
    monitor_training_accuracy=True,
    monitor_training_cost=True
       )

Epoch 0 training complete
Cost on training data: 0.425775807218
Accuracy on training data: 47004 / 50000
Cost on evaluation data: 0.48948874834
Accuracy on evaluation data: 9409 / 10000

Epoch 1 training complete
Cost on training data: 0.378368604706
Accuracy on training data: 47449 / 50000
Cost on evaluation data: 0.504470844737
Accuracy on evaluation data: 9464 / 10000

Epoch 2 training complete
Cost on training data: 0.353314331886
Accuracy on training data: 47767 / 50000
Cost on evaluation data: 0.495618246757
Accuracy on evaluation data: 9508 / 10000

Epoch 3 training complete
Cost on training data: 0.314723250755
Accuracy on training data: 48057 / 50000
Cost on evaluation data: 0.48805561381
Accuracy on evaluation data: 9566 / 10000

Epoch 4 training complete
Cost on training data: 0.305037065168
Accuracy on training data: 48178 / 50000
Cost on evaluation data: 0.502276217156
Accuracy on evaluation data: 9578 / 10000

Epoch 5 training complete
Cost on training data: 0.33096036181

In [ ]:
# Now let's compare basic and L1

L1_acc_lmbda_1 = np.asarray(L1_run_lmbda_1[1])/10000.
L1_acc_lmbda_2 = np.asarray(L1_run_lmbda_2[1])/10000.
L1_acc_lmbda_3 = np.asarray(L1_run_lmbda_3[1])/10000.
unreg_acc = np.asarray(unreg_run[1])/10000.

plt.plot(L1_acc_lmbda_1, color = "red")
plt.plot(L1_acc_lmbda_2, color = "green") #winner
plt.plot(L1_acc_lmbda_3, color = "violet")
#plt.show()
plt.plot(unreg_acc, color = "black") #baseline
plt.show()

# It seems lmbda = 2. does a little better than unregulirized run (green line)

([0.55158361190183802,
  0.55975402362235283,
  0.57982511779068369,
  0.60901525939355872,
  0.62593103125845206,
  0.65155722728092558,
  0.7060343718875457,
  0.68024698324616462,
  0.71185299196560292,
  0.72080046180677826,
  0.70726936169275234,
  0.74192230122320657,
  0.73896868909995617,
  0.73337526002067333,
  0.76578406136192867,
  0.77918724459171829,
  0.77551629298509006,
  0.85581258250230507,
  0.7890671869826531,
  0.82084515106855616,
  0.82873963191150435,
  0.8226222947445927,
  0.82687421893244384,
  0.82941507462682007,
  0.85548480832064544,
  0.84766507651435707,
  0.85284035870588792,
  0.85239202198210562,
  0.87938759816316714,
  0.87024234608500151],
 [9424,
  9528,
  9558,
  9561,
  9599,
  9590,
  9526,
  9626,
  9569,
  9564,
  9629,
  9580,
  9620,
  9644,
  9609,
  9576,
  9628,
  9530,
  9646,
  9583,
  9618,
  9615,
  9633,
  9632,
  9602,
  9647,
  9638,
  9635,
  9624,
  9623],
 [0.43166975732856527,
  0.37559335051827292,
  0.34866498926806649,
  

In [8]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = network_basic.Network([784, 30, 10], cost=network_basic.CrossEntropyCost)
net.SGD(
    training_data, 30, 10, 0.5,
    lmbda = 5.0,
    evaluation_data=validation_data,
    monitor_evaluation_accuracy=True,
    monitor_evaluation_cost=True,
    monitor_training_accuracy=True,
    monitor_training_cost=True
       )


Epoch 0 training complete
Cost on training data: 0.484534501638
Accuracy on training data: 46942 / 50000
Cost on evaluation data: 0.785656338967
Accuracy on evaluation data: 9417 / 10000

Epoch 1 training complete
Cost on training data: 0.448457462463
Accuracy on training data: 47442 / 50000
Cost on evaluation data: 0.848429661787
Accuracy on evaluation data: 9481 / 10000

Epoch 2 training complete
Cost on training data: 0.433694114602
Accuracy on training data: 47633 / 50000
Cost on evaluation data: 0.893639137586
Accuracy on evaluation data: 9509 / 10000

Epoch 3 training complete
Cost on training data: 0.42677549076
Accuracy on training data: 47749 / 50000
Cost on evaluation data: 0.908673697249
Accuracy on evaluation data: 9522 / 10000

Epoch 4 training complete
Cost on training data: 0.395013794909
Accuracy on training data: 48088 / 50000
Cost on evaluation data: 0.895941557264
Accuracy on evaluation data: 9586 / 10000

Epoch 5 training complete
Cost on training data: 0.3848816659

([0.78565633896654008,
  0.84842966178726331,
  0.89363913758649838,
  0.90867369724924285,
  0.89594155726449887,
  0.90670081159016236,
  0.92738838413209823,
  0.915660854111922,
  0.93552453716282702,
  0.9376520373429299,
  0.94978927990701423,
  0.94130182351848624,
  0.98877944661245709,
  0.93993333961107395,
  0.91943310330619776,
  0.93452307662813983,
  0.95230383636904214,
  0.93541740552140151,
  0.96047348835733581,
  0.93781170448337292,
  0.99129567068252356,
  0.93883573930967623,
  0.92944994219713417,
  1.0414957542350227,
  0.93109143844713815,
  0.92294966085897245,
  0.96046864577636271,
  0.96867640689572987,
  0.94643038707564098,
  0.95878707928444085],
 [9417,
  9481,
  9509,
  9522,
  9586,
  9580,
  9586,
  9599,
  9591,
  9598,
  9580,
  9612,
  9538,
  9601,
  9620,
  9626,
  9577,
  9612,
  9589,
  9625,
  9549,
  9622,
  9623,
  9419,
  9621,
  9634,
  9608,
  9577,
  9621,
  9585],
 [0.48453450163802214,
  0.44845746246344587,
  0.43369411460177831,
  0

In [ ]:
# Now let's compare two



In [10]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
net = network_L1.Network([784, 30, 10], cost=network_L1.CrossEntropyCost)
net.SGD(
    training_data, 30, 10, 0.5,
    lmbda = 5.0,
    evaluation_data=validation_data,
    monitor_evaluation_accuracy=True,
    monitor_evaluation_cost=True,
    monitor_training_accuracy=True,
    monitor_training_cost=True
       )


Epoch 0 training complete
Cost on training data: 0.498916795522
Accuracy on training data: 46701 / 50000
Cost on evaluation data: 0.749066915869
Accuracy on evaluation data: 9383 / 10000

Epoch 1 training complete
Cost on training data: 0.525001692245
Accuracy on training data: 46685 / 50000
Cost on evaluation data: 0.864876964999
Accuracy on evaluation data: 9340 / 10000

Epoch 2 training complete
Cost on training data: 0.458900059987
Accuracy on training data: 47353 / 50000
Cost on evaluation data: 0.860443255156
Accuracy on evaluation data: 9470 / 10000

Epoch 3 training complete
Cost on training data: 0.491539110775
Accuracy on training data: 47149 / 50000
Cost on evaluation data: 0.929947851783
Accuracy on evaluation data: 9436 / 10000

Epoch 4 training complete
Cost on training data: 0.425587902362
Accuracy on training data: 47803 / 50000
Cost on evaluation data: 0.918424344428
Accuracy on evaluation data: 9538 / 10000

Epoch 5 training complete
Cost on training data: 0.469279612

KeyboardInterrupt: 